In [78]:
import cudf
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [76]:
base_dir = 'storage/output/220317_baseline/'

****Prepare Dataset****

In [3]:
%%time 
transactions = cudf.read_csv('storage/transactions_train.csv')
articles = cudf.read_csv('storage/articles.csv')
customers = cudf.read_csv('storage/customers.csv')

CPU times: user 1.06 s, sys: 1.49 s, total: 2.55 s
Wall time: 2.56 s


In [4]:
customers['FN'].fillna(0.,inplace=True)
customers['Active'].fillna(0.,inplace=True)
customers['club_member_status'].fillna('None',inplace=True)
customers['age'] = customers['age'] / 10
customers['age'] = customers['age'].astype(int)
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].str.lower().fillna('none')

In [5]:
from utils import train_test_split
trn_transactions,test_transactions = train_test_split(transactions,gpu=True)

In [72]:
def past_purchase_feature(df,transactions):
    transactions['count'] = 1
    time_elapsed_last_purchase = transactions['t_dat'].max()-transactions[['customer_id','article_id','t_dat']].groupby(['customer_id','article_id'])['t_dat'].max()
    time_elapsed_last_purchase = time_elapsed_last_purchase.dt.days
    df = df.merge(time_elapsed_last_purchase,on=['article_id','customer_id'],how='left')
    df = df.rename(columns={'t_dat':'time_elapsed_last_purchase'})
    df['time_elapsed_last_purchase'].fillna(1e6,inplace=True)
    
    past_purchase_prob = transactions[['customer_id','article_id','count']].groupby(['customer_id','article_id'])['count'].count().reset_index()
    norm = transactions[['customer_id','article_id']].groupby('customer_id').count().reset_index().rename(columns={'article_id':'norm'})
    past_purchase_prob = past_purchase_prob.merge(norm,on='customer_id')
    past_purchase_prob['count'] = past_purchase_prob['count'] / past_purchase_prob['norm']
    past_purchase_prob.drop(columns=['norm'],inplace=True)
    df = df.merge(past_purchase_prob,on=['article_id','customer_id'],how='left')
    df = df.rename(columns={'count':'past_purchase_prob'})
    df['past_purchase_prob'].fillna(0.,inplace=True)
    
    total_purchase = transactions[['article_id','count']].groupby('article_id')['count'].count().reset_index().rename(columns={'count':'total_purchase'})
    norm = transactions['count'].sum()
    total_purchase['total_purchase'] = total_purchase['total_purchase'] / norm
    df = df.merge(total_purchase,on='article_id',how='left')
    df['total_purchase'].fillna(0.,inplace=True)
    
    number_of_purchase = transactions[['customer_id','count']].groupby('customer_id')['count'].count().reset_index().rename(columns={'count':'number_of_purchase'})
    df = df.merge(number_of_purchase,on='customer_id',how='left')
    df['number_of_purchase'].fillna(0.,inplace=True)
    
    return df
    
def article_feature_prob_vector(df,transactions,articles,article_features,postfix='_prob'):
    transactions['count'] = 1
    for article_feature in article_features:
        transactions = transactions.merge(articles[['article_id',article_feature]],on='article_id',how='left')
        norm = transactions.groupby(['customer_id'])['count'].count().reset_index()
        norm.rename(columns={'count':'norm'},inplace=True)
        count = transactions.groupby(['customer_id',article_feature])['count'].count().reset_index()
        count = count.merge(norm,on='customer_id')
        count['count'] = count['count'] / count['norm']
        count = count.rename(columns={'count':article_feature+postfix})
        count = count[['customer_id',article_feature,article_feature+postfix]]
        del(norm)
        df = df.merge(articles[['article_id',article_feature]],on='article_id',how='left')
        df = df.merge(count,on=['customer_id',article_feature],how='left')
    return df

def customer_feature_prob_vector(df,transactions,customers,customer_features,postfix='_prob'):
    transactions['count'] = 1
    for customer_feature in customer_features:
        transactions = transactions.merge(customers[['customer_id',customer_feature]],on='customer_id',how='left')
        norm = transactions.groupby(['article_id'])['count'].count().reset_index()
        norm.rename(columns={'count':'norm'},inplace=True)
        count = transactions.groupby(['article_id',customer_feature])['count'].count().reset_index()
        count = count.merge(norm,on='article_id')
        count['count'] = count['count'] / count['norm']
        count = count.rename(columns={'count':customer_feature+postfix})
        count = count[['article_id',customer_feature,customer_feature+postfix]]
        del(norm)
        df = df.merge(customers[['customer_id',customer_feature]],on='customer_id',how='left')
        df = df.merge(count,on=['article_id',customer_feature],how='left')
    return df

def construct_feature_df(
        df,transactions,
        article_features,
        articles,
        customer_features,
        customers,
        general_features=['article_id','customer_id'],
    ):
    df = article_feature_prob_vector(df,transactions,articles,article_features)
    df = customer_feature_prob_vector(df,transactions,customers,customer_features)
    df = past_purchase_feature(df,transactions)
    df = df[general_features+[f for f in df.columns if '_prob' in f]+['time_elapsed_last_purchase','past_purchase_prob','number_of_purchase']]
    return df

def construct_candidate_dict(transactions_3w):
    purchase_dict_3w = {}
    for i,x in enumerate(zip(transactions_3w['customer_id'], transactions_3w['article_id'])):
        cust_id, art_id = x
        if cust_id not in purchase_dict_3w:
            purchase_dict_3w[cust_id] = {}
        if art_id not in purchase_dict_3w[cust_id]:
            purchase_dict_3w[cust_id][art_id] = 0
        purchase_dict_3w[cust_id][art_id] += 1
    dummy_list_3w = list((transactions_3w['article_id'].value_counts()).index)[:12]
    return purchase_dict_3w,dummy_list_3w

def construct_candidate_df(test_df,transactions,add_random_samples=False):
    
    bool_1w = transactions.t_dat>transactions.t_dat.max()-pd.Timedelta(7,unit='day')
    bool_2w = (transactions.t_dat>transactions.t_dat.max()-2*pd.Timedelta(7,unit='day'))&(transactions.t_dat<=transactions.t_dat.max()-pd.Timedelta(7,unit='day'))
    bool_3w = (transactions.t_dat>transactions.t_dat.max()-3*pd.Timedelta(7,unit='day'))&(transactions.t_dat<=transactions.t_dat.max()-2*pd.Timedelta(7,unit='day'))
    
    transactions_1w = transactions[bool_1w].to_pandas()
    transactions_2w = transactions[bool_2w].to_pandas()
    transactions_3w = transactions[bool_3w].to_pandas()
    
    purchase_dict_1w,dummy_list_1w = construct_candidate_dict(transactions_1w)
    purchase_dict_2w,dummy_list_2w = construct_candidate_dict(transactions_2w)
    purchase_dict_3w,dummy_list_3w = construct_candidate_dict(transactions_3w)
    
    pred_df = test_df[['customer_id']]
    prediction_list = []
    
    if add_random_samples:
        dummy_pred = transactions['article_id'].sample(frac=1.).to_arrow().to_pylist()[:50]
    else:
        dummy_pred = list((transactions_1w['article_id'].value_counts()).index)[:12]
    
    for i, cust_id in enumerate(test_df['customer_id'].values.reshape((-1,))):
        s = []
        if cust_id in purchase_dict_1w:
            l = sorted((purchase_dict_1w[cust_id]).items(), key=lambda x: x[1], reverse=True)
            l = [y[0] for y in l]
            s += l
        if cust_id in purchase_dict_2w:
            l = sorted((purchase_dict_2w[cust_id]).items(), key=lambda x: x[1], reverse=True)
            l = [y[0] for y in l]
            s += l
        if cust_id in purchase_dict_3w:
            l = sorted((purchase_dict_3w[cust_id]).items(), key=lambda x: x[1], reverse=True)
            l = [y[0] for y in l]
            s += l
        s += dummy_pred
        s = list(set(s))
        prediction_list.append(s)
    pred_df['article_id'] = prediction_list

    return pred_df
    
def construct_val_df(test_df,transactions,article_features,articles,customer_features,customers,how='outer',add_random_samples=False):
    pos_df = test_df.groupby('customer_id')['article_id'].unique().to_frame().reset_index().explode('article_id')
    pos_df['label'] = 1
    test_df = construct_candidate_df(test_df.to_pandas(),transactions,add_random_samples=add_random_samples).explode('article_id').reset_index(drop=True)
    test_df = test_df.merge(pos_df.to_pandas(),on=['article_id','customer_id'],how=how)
    test_df['label'].fillna(0,inplace=True)
    test_df = cudf.from_pandas(test_df)
    test_df = construct_feature_df(test_df,transactions,article_features,articles,customer_features,customers,general_features=['article_id','customer_id','label'])
    test_df = test_df.fillna(0.)
    test_df['article_id'] = test_df['article_id'].astype(int)
    test_df = test_df.sort_values(['customer_id','article_id']).reset_index(drop=True)
    return test_df

def construct_test_df(test_df,transactions,article_features,articles,customer_features,customers,how='outer',add_random_samples=False):
    test_df = construct_candidate_df(test_df.to_pandas(),transactions,add_random_samples=add_random_samples).explode('article_id').reset_index(drop=True)
    test_df = cudf.from_pandas(test_df)
    test_df = construct_feature_df(test_df,transactions,article_features,articles,customer_features,customers,general_features=['article_id','customer_id'])
    test_df = test_df.fillna(0.)
    test_df['article_id'] = test_df['article_id'].astype(int)
    test_df = test_df.sort_values(['customer_id','article_id']).reset_index(drop=True)
    return test_df

def construct_gt_df(test_transactions):
    gt_df = test_transactions.to_pandas().groupby('customer_id')['article_id'].agg(lambda x: x.tolist()).reset_index()
    gt_df.columns = ['customer_id','ground_truth']
    return gt_df
    
def construct_dataset(
        transactions,
        articles,customers,
        trn_start_time='2020-08-31',trn_end_time='2020-09-08',
        val_start_time='2020-09-08',val_end_time='2020-09-15',
        test_start_time='2020-09-08',test_end_time='2020-09-15',
        article_features=[
            'product_group_name', 'product_type_name', 
            'graphical_appearance_name', 'perceived_colour_value_name', 'colour_group_code', 
            'index_name', 'index_group_name', 
            'section_name', 'department_name',
        ],
        customer_features=[
            'FN','Active','club_member_status','age','fashion_news_frequency',
        ],
    ):
    
    trn_start_time = cudf.to_datetime(trn_start_time)
    trn_end_time = cudf.to_datetime(trn_end_time)
    val_start_time = cudf.to_datetime(val_start_time)
    val_end_time = cudf.to_datetime(val_end_time)
    test_start_time = cudf.to_datetime(test_start_time)
    test_end_time = cudf.to_datetime(test_end_time)
    
    trn_transactions = transactions[(transactions.t_dat > trn_start_time) & (transactions.t_dat <= trn_end_time)]
    val_transactions = transactions[(transactions.t_dat > val_start_time) & (transactions.t_dat <= val_end_time)]
    test_transactions = transactions[(transactions.t_dat > test_start_time) & (transactions.t_dat <= test_end_time)]
    gt_df = construct_gt_df(test_transactions)
    
    pos_df = val_transactions[['customer_id','article_id']]
    pos_df = construct_feature_df(pos_df,trn_transactions,article_features,articles,customer_features,customers)
    pos_df['label'] = 1.
    
    trn_dfs = [pos_df]
    for _ in range(5):
        neg_df = val_transactions[['customer_id','article_id']].reset_index().drop(columns=['index'])
        neg_df['customer_id'] = neg_df['customer_id'].sample(frac=1.).to_frame().reset_index().drop(columns=['index'])
        neg_df = construct_feature_df(neg_df,trn_transactions,article_features,articles,customer_features,customers)
        neg_df['label'] = 0.
        trn_dfs.append(neg_df)
    
    trn_df = cudf.concat(trn_dfs)
    trn_df = trn_df.fillna(0.)
    trn_df = trn_df.sort_values(['customer_id','article_id']).reset_index(drop=True)
    
    trn_df = trn_df.merge(trn_df.groupby('customer_id').size().to_frame().rename(columns={0:'group_size'}),on='customer_id')
    trn_df = trn_df[trn_df['group_size']>5]
    
    #val_df = construct_test_df(test_transactions,val_transactions,article_features,articles,customer_features,customers,how='left')
    test_df = construct_test_df(test_transactions,val_transactions,article_features,articles,customer_features,customers,how='left')
    val_df = test_df
    
    return trn_df.reset_index(drop=True),val_df.reset_index(drop=True),test_df.reset_index(drop=True),gt_df.reset_index(drop=True)
    

In [ ]:
dfs = []
for i,(t1,t2,t3,t4) in enumerate([
        ('2020-06-01','2020-09-01','2020-09-15','2020-09-22'),
        #('2019-06-01','2019-09-01','2019-09-15','2019-09-22'),
    ]):
    trn_tmp,val_tmp,test_tmp,gt_tmp = construct_dataset(
        transactions,
        articles,customers,
        trn_start_time=t1,trn_end_time=t2,
        val_start_time=t2,val_end_time=t3,
        test_start_time=t3,test_end_time=t4,
    )
    dfs.append((trn_tmp,val_tmp,test_tmp,gt_tmp))

****Training****

In [23]:
import xgboost as xgb

In [46]:
best_hyperparams = None

In [47]:
def x_y_group(data,features,target,only_x=False,verbose=False):
    group = data.groupby('customer_id').size().to_frame('size')['size']
    data = data.sort_values('customer_id').reset_index()
    return data[features],data[target],group

def make_prediction(model,test_df,features,label,k=12,group_name='customer_id'):
    test_x = test_df[features]
    test_pred = model.predict(test_x)
    test_x[group_name] = test_df[group_name]
    test_x['article_id'] = test_df['article_id']
    test_x['prediction'] = test_pred
    pred_df = test_x.groupby(group_name) \
                    .apply(lambda x: x.sort_values('prediction',ascending=False)['article_id'].tolist()[:k]) \
                    .reset_index()
    pred_df.columns = [group_name,'prediction']
    return pred_df

def evaluate_score(pred_df,gt_df,k=12,verbose=True,group_name='customer_id'):
    from metric import mapk
    eval_df = gt_df.merge(pred_df,on=group_name,how='left')
    score = mapk(eval_df['ground_truth'].tolist(),eval_df['prediction'].tolist())
    if verbose: print('map@'+str(k),score)
    return score

In [48]:
%%time
features = [c for c in dfs[0][0].columns if c not in ['article_id','customer_id','label','index','group_size']]
label = 'label'
#trn_x,trn_y,trn_grp = x_y_group(trn_df,features,label)
#val_x,val_y,val_grp = x_y_group(val_df,features,label)

CPU times: user 190 µs, sys: 0 ns, total: 190 µs
Wall time: 194 µs


In [49]:
%%time
run_bo = True
if run_bo and best_hyperparams is None:
    from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

    space = {
        'objective': 'rank:pairwise',
        'max_depth': hp.choice("max_depth",np.arange(3, 20, dtype=int) ),
        'gamma': hp.uniform ('gamma', 1,9),
        'learning_rate': hp.uniform('learning_rate',0.1,1.0),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        #'n_estimators': hp.choice("n_estimators",np.arange(3, 10, dtype=int) ),
        'n_estimators': 5,
        'seed': 0,
        'eval_metric':'map@12',
    }

    def time_fold_objective(space):
        scores = []
        for trn_df,val_df,test_df,gt_df in dfs:
            model = xgb.XGBRanker(**space)
            trn_x,trn_y,trn_grp = x_y_group(trn_df,features,label)
            model.fit(trn_x, trn_y, trn_grp)
            pred_df = make_prediction(model,test_df.to_pandas(),features,label,k=12)
            score = evaluate_score(pred_df,gt_df,verbose=False)
            scores.append(score)
        print(space)
        print(scores)
        return {'loss': 1.-np.mean(scores),'status': STATUS_OK}

    trials = Trials()
    best_hyperparams = fmin(
        fn = time_fold_objective,
        space = space,
        algo = tpe.suggest,
        max_evals = 5,
        trials = trials
    )
    print('Best hyperparameters: ',best_hyperparams)

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

/home/kinho.lo/.local/lib/python3.8/site-packages/xgboost/core.py:499: FutureWarning: Pass `group` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(

/scratch/local/21399652/ipykernel_193925/4083372006.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x[group_name] = test_df[group_name]



{'colsample_bytree': 0.6842720483155474, 'eval_metric': 'map@12', 'gamma': 4.621670610082947, 'learning_rate': 0.7435578636949653, 'max_depth': 13, 'min_child_weight': 1.0, 'n_estimators': 5, 'objective': 'rank:pairwise', 'reg_alpha': 98.0, 'reg_lambda': 0.8902384952199381, 'seed': 0}
[0.01753785424273931]                                
{'colsample_bytree': 0.877948622970067, 'eval_metric': 'map@12', 'gamma': 3.4873495479305046, 'learning_rate': 0.20337506434029795, 'max_depth': 3, 'min_child_weight': 3.0, 'n_estimators': 5, 'objective': 'rank:pairwise', 'reg_alpha': 133.0, 'reg_lambda': 0.707379435453157, 'seed': 0}
[0.017784135119975675]                                                         
{'colsample_bytree': 0.8697290277349912, 'eval_metric': 'map@12', 'gamma': 7.833981406446694, 'learning_rate': 0.8974579001210463, 'max_depth': 3, 'min_child_weight': 7.0, 'n_estimators': 5, 'objective': 'rank:pairwise', 'reg_alpha': 47.0, 'reg_lambda': 0.1730920167886617, 'seed': 0}
[0.017852

In [52]:
%%time
idx = 0
trn_x,trn_y,trn_grp = x_y_group(dfs[idx][0],features,label)
model = xgb.XGBRanker(
    objective='rank:pairwise',
    seed=0,
    n_estimators=5,
    **best_hyperparams,
)
model.fit(
    trn_x, trn_y, group=trn_grp, verbose=True,
    eval_set=[(trn_x,trn_y)], eval_group=[trn_grp],
)

[0]	validation_0-map:0.48883
[1]	validation_0-map:0.49711
[2]	validation_0-map:0.49884
[3]	validation_0-map:0.49935
[4]	validation_0-map:0.50350
CPU times: user 1min 3s, sys: 321 ms, total: 1min 3s
Wall time: 16.7 s


XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=0.80186147510322,
          enable_categorical=False, gamma=3.105133146859485, gpu_id=-1,
          importance_type=None, interaction_constraints='',
          learning_rate=0.6935952892317561, max_delta_step=0, max_depth=11,
          min_child_weight=4.0, missing=nan, monotone_constraints='()',
          n_estimators=5, n_jobs=4, num_parallel_tree=1, predictor='auto',
          random_state=0, reg_alpha=95.0, reg_lambda=0.7071059022165264,
          scale_pos_weight=None, seed=0, subsample=1, tree_method='exact',
          validate_parameters=1, verbosity=None)

In [53]:
model.get_booster().get_score(importance_type='gain')

{'product_group_name_prob': 28.817983627319336,
 'product_type_name_prob': 75.40428161621094,
 'graphical_appearance_name_prob': 27.07487678527832,
 'perceived_colour_value_name_prob': 50.96394348144531,
 'colour_group_code_prob': 41.485050201416016,
 'index_name_prob': 62.60843276977539,
 'index_group_name_prob': 38.46046447753906,
 'section_name_prob': 455.3945007324219,
 'department_name_prob': 531.75244140625,
 'FN_prob': 22.918670654296875,
 'Active_prob': 27.318485260009766,
 'club_member_status_prob': 28.38362693786621,
 'age_prob': 144.56809997558594,
 'fashion_news_frequency_prob': 20.809524536132812,
 'past_purchase_prob': 4505.01513671875,
 'time_elapsed_last_purchase': 431.93896484375,
 'number_of_purchase': 67.66783905029297}

****Local CV****

In [54]:
%%time
pred_df = make_prediction(model,dfs[idx][2].to_pandas(),features,label,k=12)

/scratch/local/21399652/ipykernel_193925/4083372006.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x[group_name] = test_df[group_name]


CPU times: user 16.2 s, sys: 675 ms, total: 16.9 s
Wall time: 15.7 s


In [55]:
evaluate_score(
    pred_df,
    dfs[idx][-1],
)

map@12 0.017833652590270584


0.017833652590270584

****Submission****

In [71]:
%%time
article_features=[
    'product_group_name', 'product_type_name', 
    'graphical_appearance_name', 'perceived_colour_value_name', 'colour_group_code', 
    'index_name', 'index_group_name', 
    'section_name', 'department_name',
]
customer_features=[
    'FN','Active','club_member_status','age','fashion_news_frequency',
    ]
submission_df = cudf.read_csv('storage/sample_submission.csv')
submission_df = construct_test_df(
    submission_df[['customer_id']],
    transactions[(transactions.t_dat > cudf.to_datetime('2020-09-07')) & (transactions.t_dat <= cudf.to_datetime('2020-09-22'))],
    article_features,articles,customer_features,customers,
    how='left',
)
submission_df = make_prediction(model,submission_df.to_pandas(),features,label,k=12)

['product_group_name', 'product_type_name', 'graphical_appearance_name', 'perceived_colour_value_name', 'colour_group_code', 'index_name', 'index_group_name', 'section_name', 'department_name']


/scratch/local/21399652/ipykernel_193925/4083372006.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x[group_name] = test_df[group_name]


In [74]:
submission_df['prediction'] = submission_df['prediction'].apply(lambda x: ' '.join(['0'+str(i) for i in x]))

In [82]:
os.makedirs(base_dir,exist_ok=True)
submission_df.to_csv(os.path.join(base_dir,'submission.csv'),index=False)

In [81]:
submission_df

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0751471001 0866731001 0448509014 0762846027 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0448509014 0762846027 0866731001 0915529003 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0866731001 0915529005 0714790020 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0762846027 0751471001 0924243001 0918522001 04...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0751471001 0762846027 0918522001 0924243001 09...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0448509014 0762846027 0866731001 0915529003 07...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0448509014 0762846027 0866731001 0915529003 07...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0762846027 0884081001 0689365050 0794819001 09...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0751471001 0866731001 0918522001 0714790020 04...
